# Proyecto de visualización de datos (Parte II)

### Limpieza del dataset (fifa21.csv)

**Autor:** Fernando José Cofiño Gavito

### 0. Carga de librerías y dataset

In [1]:
# Librerías y dependencias
import pandas as pd
import numpy as np
import os

In [2]:
# Cargar el dataset
data = pd.read_csv('fifa21.csv', low_memory=False)

In [3]:
# Mostrar cabecera del dataset inicial
data.head()

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,33,93,93,\n\n\n\nFC Barcelona,...,Medium,Low,5 ★,85,92,91,95,38,65,771
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,35,92,92,\n\n\n\nJuventus,...,High,Low,5 ★,89,93,81,89,35,77,562
2,200389,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,\n\n\n\nAtlético Madrid,...,Medium,Medium,3 ★,87,92,78,90,52,90,150
3,192985,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,29,91,91,\n\n\n\nManchester City,...,High,High,4 ★,76,86,93,88,64,78,207
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,28,91,91,\n\n\n\nParis Saint-Germain,...,High,Medium,5 ★,91,85,86,94,36,59,595


### 1. Eliminar columnas no deseadas

In [4]:
# Detalle de columnas que vamos a borrar
columns_to_drop = ['ID', 'playerUrl', 'Attacking', 'Skill', 'Movement', 'Power', 'Mentality', 'Defending', 
                   'Goalkeeping', 'Total Stats', 'Base Stats', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'Hits']

# Borrado de las columnas
data.drop(columns=columns_to_drop, inplace=True, errors='ignore')

### 2. Crear nuevas columnas a partir de ("Club", "Joined", "Contract", "Loan Date End")

In [5]:
# Función para determinar el estado contractual del jugador
def determine_player_state(row):
    if row['Club'] == 'No Club':
        return 'Libre'
    elif pd.notna(row['Loan Date End']):
        return 'Cedido'
    else:
        return 'Con Contrato'

# Cálculo del estado contractual del jugador
data['playerState'] = data.apply(determine_player_state, axis=1)

In [6]:
# Función para determinar la fecha de inico del contrato del jugador
def extract_date_start_contract(row):
    if row['playerState'] == 'Libre':
        return np.nan
    if isinstance(row['Joined'], str):
        try:
            return pd.to_datetime(row['Joined'])
        except ValueError:
            return np.nan
    return np.nan

# Cálculo de la fecha de inicio del contrato del jugador
data['dateStartContract'] = data.apply(extract_date_start_contract, axis=1)

In [7]:
# Función para determinar la fecha de fin del contrato del jugador
def extract_date_end_contract(row):
    if row['playerState'] == 'Cedido':
        if isinstance(row['Loan Date End'], str):
            try:
                return pd.to_datetime(row['Loan Date End'])
            except ValueError:
                return np.nan
    elif row['playerState'] == 'Con Contrato':
        if isinstance(row['Contract'], str) and '~' in row['Contract']:
            try:
                end_year = row['Contract'].split('~')[-1].strip()
                return pd.to_datetime(f"{end_year}-06-30")
            except ValueError:
                return np.nan
    return np.nan

# Cálculo de la fecha de fin del contrato del jugador
data['dateEndContract'] = data.apply(extract_date_end_contract, axis=1)

In [8]:
# Limpieza de la columna "Club"
data['Club'] = data['Club'].str.replace(r'\\n', '', regex=True).str.strip()

In [9]:
# Eliminar columnas antiguas
data.drop(columns=['Contract', 'Joined', 'Loan Date End'], inplace=True, errors='ignore')

### 3. Convertir "Height" a cm

In [10]:
# Función para convertir las alturas de los jugadores en un número entero expresado en cm
def convert_height(value):
    if isinstance(value, str):
        if 'cm' in value:
            return int(value.replace('cm', ''))
        elif "'" in value:
            try:
                feet, inches = value.split("'")
                feet = int(feet)
                inches = int(inches.replace('"', ''))
                return round(feet * 30.48 + inches * 2.54)
            except ValueError:
                return np.nan
    return np.nan

# Cálculo de las alturas de los jugadores
data['heightCm'] = data['Height'].apply(convert_height)
data.drop(columns=['Height'], inplace=True, errors='ignore')

### 4. Convertir "Weight" a kg

In [11]:
# Función para convertir los pesos de los jugadores en un número entero expresado en kg
def convert_weight(value):
    if isinstance(value, str):
        if 'kg' in value:
            return int(value.replace('kg', ''))
        elif 'lbs' in value:
            return round(int(value.replace('lbs', '')) * 0.453592)
    return np.nan

# Cálculo de los pesos de los jugadores
data['weightKg'] = data['Weight'].apply(convert_weight)
data.drop(columns=['Weight'], inplace=True, errors='ignore')

### 5. Convertir ("Value", "Wage", "Release Clause") a números enteros

In [12]:
# Función para convertir las variables económicas a formato int
def convert_currency(value):
    if isinstance(value, str):
        value = value.replace('€', '').replace(' ', '')
        if 'M' in value:
            value = value.replace('M', '')
            return int(float(value) * 1_000_000)
        elif 'K' in value:
            value = value.replace('K', '')
            return int(float(value) * 1_000)
        else:
            return 0
    return 0

# Conversión de las variables económicas a formato int
for col in ['Value', 'Wage', 'Release Clause']:
    data[col] = data[col].apply(convert_currency)

### 6. Normalizar ("W/F", "SM", "IR", "A/W", "D/W")

In [13]:
# Funciones para normalizar distintas variables en el rango [0, 99]
def normalize_stars(value):
    mapping = {'5 ★': 99, '4 ★': 80, '3 ★': 60, '2 ★': 40, '1 ★': 20}
    return mapping.get(value, np.nan)
def normalize_stars_no_spaces(value):
    mapping = {'5★': 99, '4★': 80, '3★': 60, '2★': 40, '1★': 20}
    return mapping.get(value, np.nan)
def normalize_workrate(value):
    mapping = {'High': 99, 'Medium': 67, 'Low': 33}
    return mapping.get(value, np.nan)

# Normalización de variables en el rango [0, 99]
for col in ['W/F', 'IR']:
    data[col] = data[col].apply(normalize_stars)
for col in ['SM']:
    data[col] = data[col].apply(normalize_stars_no_spaces)
for col in ['A/W', 'D/W']:
    data[col] = data[col].apply(normalize_workrate)

### 7. Traducir "Positions" y "Best Position" (siglas ESP)

In [14]:
# Traducciones de siglas de posición (ENG -> ESP)
translation = {
    'GK': 'POR', 'RWB': 'CAD', 'RB': 'LTD', 'CB': 'DFC', 'LB': 'LTI',
    'LWB': 'CAI', 'CDM': 'MCD', 'RM': 'MD', 'CM': 'MC', 'LM': 'MI',
    'CAM': 'MCO', 'RF': 'SDD', 'CF': 'MP', 'LW': 'EI', 'ST': 'DC', 'RW': 'ED'
}

# Función para traducir las siglas de posición
def translate_positions(value):
    if isinstance(value, str):
        positions = value.split(', ')
        return ', '.join([translation.get(pos, pos) for pos in positions])
    return value

# Traducción de las siglas de posición (ENG -> ESP)
for col in ['Positions', 'Best Position']:
    data[col] = data[col].apply(translate_positions)

### 8. Renombrar columnas

In [15]:
# Renombrar columnas para unificar formatos
data.rename(columns={'Name': 'playerName', 'LongName': 'playerLongName', 'photoUrl': 'photoUrl', 'Club': 'club',
                     'Nationality': 'nationality', 'Age': 'age', '↓OVA': 'overallRating', 'POT': 'potential', 
                     'Positions': 'positions', 'Height': 'height', 'Weight': 'weight',
                     'Preferred Foot': 'preferredFoot', 'BOV': 'bestOverallRating', 'Best Position': 'bestPosition', 
                     'Value': 'value', 'Wage': 'wage','Release Clause': 'releaseClause', 'Crossing': 'crossing', 
                     'Finishing': 'finishing','Heading Accuracy': 'heading', 'Short Passing': 'shortPassing', 
                     'Volleys': 'volleys', 'Dribbling': 'dribbling', 'Curve': 'curve', 
                     'FK Accuracy': 'freeKick','Long Passing': 'longPassing', 'Ball Control': 'ballControl',
                     'Acceleration': 'acceleration', 'Sprint Speed': 'sprintSpeed', 'Agility': 'agility', 
                     'Reactions': 'reactions', 'Balance': 'balance', 'Shot Power': 'shotPower', 'Jumping': 'jumping',
                     'Stamina': 'stamina', 'Strength': 'strength', 'Long Shots': 'longShots',
                     'Aggression': 'aggression', 'Interceptions': 'interceptions', 'Positioning': 'positioning',
                     'Vision': 'vision', 'Penalties': 'penalties', 'Composure': 'composure', 'Marking': 'marking', 
                     'Standing Tackle': 'standingTackle', 'Sliding Tackle': 'slidingTackle','GK Diving': 'gkDiving', 
                     'GK Handling': 'gkHandling', 'GK Kicking': 'gkKicking', 'GK Positioning': 'gkPositioning', 
                     'GK Reflexes': 'gkReflexes', 'W/F': 'weakFoot', 'SM': 'skillMoves', 'IR': 'reputation',
                     'A/W': 'attackingWork', 'D/W': 'defensiveWork',
}, inplace=True)

### 9. Últimas verificaciones

In [16]:
# Verificar valores nulos
nulls = data.isnull().sum()
empty_columns = nulls[nulls > 0]

# Mostrar resumen de nulos
print("Columnas con valores nulos:")
print(empty_columns)

Columnas con valores nulos:
dateStartContract    237
dateEndContract      237
dtype: int64


Los 237 nulos en las columnas "dateEndContract" y "dateStartContract" se corresponden con jugadores libres. Se entiende que es correcto que si están sin contrato no tengan fecha de expiración o inicio del contrato.

In [17]:
# Verificar nombre de columnas
print(data.columns)

Index(['playerName', 'playerLongName', 'photoUrl', 'nationality', 'age',
       'overallRating', 'potential', 'club', 'positions', 'preferredFoot',
       'bestOverallRating', 'bestPosition', 'value', 'wage', 'releaseClause',
       'crossing', 'finishing', 'heading', 'shortPassing', 'volleys',
       'dribbling', 'curve', 'freeKick', 'longPassing', 'ballControl',
       'acceleration', 'sprintSpeed', 'agility', 'reactions', 'balance',
       'shotPower', 'jumping', 'stamina', 'strength', 'longShots',
       'aggression', 'interceptions', 'positioning', 'vision', 'penalties',
       'composure', 'marking', 'standingTackle', 'slidingTackle', 'gkDiving',
       'gkHandling', 'gkKicking', 'gkPositioning', 'gkReflexes', 'weakFoot',
       'skillMoves', 'attackingWork', 'defensiveWork', 'reputation',
       'playerState', 'dateStartContract', 'dateEndContract', 'heightCm',
       'weightKg'],
      dtype='object')


In [18]:
# Verificar cabecera del dataset final
data.head()

,playerName,playerLongName,photoUrl,nationality,age,overallRating,potential,club,positions,preferredFoot,...,weakFoot,skillMoves,attackingWork,defensiveWork,reputation,playerState,dateStartContract,dateEndContract,heightCm,weightKg
0,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,Argentina,33,93,93,FC Barcelona,"ED, DC, MP",Left,...,80,80,67,33,99,Con Contrato,2004-07-01,2021-06-30,170,72
1,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,Portugal,35,92,92,Juventus,"DC, EI",Right,...,80,99,99,33,99,Con Contrato,2018-07-10,2022-06-30,187,83
2,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,Slovenia,27,91,93,Atlético Madrid,POR,Right,...,60,20,67,67,60,Con Contrato,2014-07-16,2023-06-30,188,87
3,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,Belgium,29,91,91,Manchester City,"MCO, MC",Right,...,99,80,99,99,80,Con Contrato,2015-08-30,2023-06-30,181,70
4,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,Brazil,28,91,91,Paris Saint-Germain,"EI, MCO",Right,...,99,99,99,67,99,Con Contrato,2017-08-03,2022-06-30,175,68


### 10. Guardar el dataset limpio

In [20]:
# Guardar el dataset después de la limpieza
data.to_csv('fifa21_clean.csv', index=False)

# Mostrar mensaje de que se ha completado con éxito
print("Dataset limpiado y guardado como 'fifa21_clean.csv'.")

Dataset limpiado y guardado como 'fifa21_clean.csv'.


### 11. Conclusiones finales

El resultado es un dataset optimizado, con columnas descriptivas y datos limpios que nos facilitarán la comprensión de la información. Esto también nos permitirá realizar análisis más efectivos sobre las habilidades, contratos y posiciones de los jugadores... que completarán nuestra visualización final.